## Resume Parser 

### 1. Data Gathering and Processing

To create a diverse and representative dataset for this project, I gathered a small set of **7 synthetic resumes** in both PDF and Word formats. This approach ensures the solution is not overfitted to any specific resume style and can handle variations in document structure. This small sample will also be used for evaluating the project's performance.

The resume samples were sourced from the following platforms:

* **Indeed Resume Builder** ([www.resume.com](https://www.resume.com)): I created and downloaded **3 resumes in PDF format** using this free tool. The profile information (names, emails, and skills) was generated using an AI tool (Gemini 2.5 Flash) to ensure the data was fictitious and unique to this project.
* **Microsoft 365 Resume Templates** ([create.microsoft.com/en-us/templates/resumes](https://create.microsoft.com/en-us/templates/resumes)): To obtain samples in the **Word (.docx) format**, I utilized the free resume template bank provided by Microsoft. I created and downloaded **4 resumes in .docx format**. These templates cover a range of professional layouts and designs, which helps in testing the robustness of the parsing algorithm across different document structures.

By using these methods, I was able to create a small but effective dataset to develop and evaluate the resume parser.

### 2. Proof of Concept (POC) and System Design

The development of this resume parser will follow a **Test-Driven Development (TDD)** approach. This means we will start by writing tests that specify the expected behavior of our solution. These tests will serve as our development roadmap, ensuring that our code correctly handles the required inputs and produces the desired structured output.

---

#### 2.1. Overall System Pipeline

The core of this project is a robust pipeline designed to extract structured information from various resume file formats. The proposed system design is a multi-step process that leverages specialized libraries for document conversion and the power of a large language model (LLM) for intelligent information extraction.

The high-level pipeline can be summarized as follows:

1.  **TDD Driver**: The development begins with test cases written in `pytest`, which define the expected `name`, `email`, and `skills` for our sample resumes.
2.  **Document Ingestion**: The system accepts resumes in either PDF or Word (.docx) format.
3.  **Document-to-Markdown Conversion**: Both file types are converted into a standardized Markdown format. This is a critical step, as Markdown preserves key structural elements (like headings, lists, and bold text) while creating a clean, text-based representation. This format is also ideal for processing by LLMs, as they are often pre-trained on a vast corpus of Markdown text, leading to more predictable and accurate results.
4.  **LLM-based Information Extraction**: The Markdown text of the resume is passed to a powerful LLM with a carefully crafted prompt. The prompt directs the LLM to identify and extract the candidate's name, email, and skills.
5.  **Structured Output Generation**: The LLM is configured to return the extracted information in a structured JSON format, ensuring a clean and consistent output.
6.  **Output Validation**: The final JSON object is validated against a defined schema, and the test cases verify that the output matches the ground truth.

---

#### 2.2. Technical Stack and Tooling

To implement this pipeline, I propose using a specific set of tools and packages for each step, balancing efficiency, cost, and maintainability.

**Testing:**

* The project will be **test-driven using `pytest`**. Tests will be written to fail initially, guiding the implementation of the parsing logic until they pass. This ensures all functionality is verified and provides a safety net for future refactoring.

**Document Conversion:**

* **Word (.docx) to Markdown**: The `markdown-it-doc` package is the primary choice for its reliability in converting Microsoft Word documents to Markdown. As a fallback, `pandoc` can be used, a versatile command-line tool known for its extensive support for various document conversions.
* **PDF to Markdown**: For PDF parsing, `pymupdf4llm` will be leveraged. This library is specifically designed to handle the complexities of PDF layouts and produce LLM-friendly output. It provides a more robust solution compared to general-purpose PDF-to-text libraries. A simple fallback would be `PyMuPDF`, which offers basic text extraction capabilities.

**LLM and Prompt Engineering:**

* **LLM Service**: The API from [OpenRouter](https://openrouter.ai/) will be used to access various LLMs, prioritizing **`Gpt4-mini`** due to its low cost and strong performance. This choice balances accuracy with cost-effectiveness.
* **Structured Output**: A key challenge with the OpenRouter API is the lack of explicit, well-documented support for structured output formats (like a JSON schema). To address this, a robust method will be implemented using a combination of a well-designed prompt and the Python `pydantic` library.
    * **Prompting**: The LLM will be given clear instructions to return the output in a specific JSON format, explicitly listing the keys (`name`, `email`, `skills`) and their corresponding data types.
    * **Pydantic**: The `pydantic` library will be used to define a data model that mirrors the required JSON schema. The LLM's output will be passed to this model, which will handle the parsing and validation. This approach ensures the final output is always in the correct format, handles potential parsing errors gracefully, and provides clear type-hinting for clean code.

**Project Management and Development Practices:**

* **Environment Management**: `uv` will be used for project and dependency management.
* **Version Control**: Git will be used for source control, with the project hosted on a private GitHub repository for development, which will be made public upon completion to satisfy the project requirements.


In [ ]:
def test_resume_parser():
    """Simple TDD test function with multiple test cases"""
    parser = ResumeParser()
    
    # Test case 1: Classic management resume (DOCX)
    print("Testing Classic management resume...")
    result1 = parser("./data/inputs/Classic management resume.docx")
    expected1 = {
        "name": "Carmelo Barese",
        "email": "carmelo@example.com",
        "skills": ["Marketing", "Communication", "Project management", "Problem-solving", "Budget planning"]
    }
    
    assert result1["name"] == expected1["name"], f"Test 1 - Name: Expected {expected1['name']}, got {result1['name']}"
    assert result1["email"] == expected1["email"], f"Test 1 - Email: Expected {expected1['email']}, got {result1['email']}"
    assert set(result1["skills"]) == set(expected1["skills"]), f"Test 1 - Skills mismatch"
    print("✅ Test 1 passed!")
    
    # Test case 2: Customer Production Assistant resume (PDF)
    print("Testing Customer Production Assistant resume...")
    result2 = parser("data/inputs/Customer Production Assistant Resume.pdf")
    expected2 = {
        "name": "Jessica Garcia",
        "email": "jessgarcia@gmail.com",
        "skills": ["Microsoft Office", "Basic Math", "Communication Skills", "Microsoft Outlook", "Manufacturing", "Computer Skills", "Microsoft Excel", "Education"]
    }
    
    assert result2["name"] == expected2["name"], f"Test 2 - Name: Expected {expected2['name']}, got {result2['name']}"
    assert result2["email"] == expected2["email"], f"Test 2 - Email: Expected {expected2['email']}, got {result2['email']}"
    assert set(result2["skills"]) == set(expected2["skills"]), f"Test 2 - Skills mismatch"
    print("✅ Test 2 passed!")
    
    print("🎉 All tests passed! Both DOCX and PDF parsing work correctly.")

class ResumeParser:
    def __call__(self, filepath):
        # Implementation will go here - starts with failing test
        return {"name": "Carmelo Barese", "email": None, "skills": []}


In [11]:
test_resume_parser()

Testing Classic management resume...


AssertionError: Test 1 - Name: Expected Carmelo Barese, got None

In [ ]:
from markitdown import MarkItDown

md = MarkItDown(enable_plugins=False) # Set to True to enable plugins
result = md.convert("test.xlsx")
print(result.text_content)